### Exploratory Analysis of Covid-19 Cases, Hospitalizations, and Hospital Capacity

#### Aaron McAdie

I've been really curious about how the current case load in the US compares with regional hospital beds, and which regions are approaching their capacity.  The folks at the New York times just open sourced their [county-level Covid-19 case dataset](https://github.com/nytimes/covid-19-data), and it provides a great jumping off point to probe the question.  It's also a good opportunity to brush up on Python, since I do ~90% of my day job using R.

Also, since I started working on this, I came across the [incredible modeling and data viz](https://covid19.healthdata.org/projections) done by the UW IHME team.  They have done a way better job than I have the subject matter expertise to match, so I'll pull in their projections and explore it from a few different angles.

In [1]:
import requests
import re
import zipfile
from bs4 import BeautifulSoup
import pandas as pd
import altair as alt

#### Dataset 1 - NYTimes Cases, thanks NYT!

In [2]:
cases = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')

In [3]:
cases.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0
1,2020-01-22,Snohomish,Washington,53061.0,1,0
2,2020-01-23,Snohomish,Washington,53061.0,1,0
3,2020-01-24,Cook,Illinois,17031.0,1,0
4,2020-01-24,Snohomish,Washington,53061.0,1,0


In [4]:
cases.dtypes

date       object
county     object
state      object
fips      float64
cases       int64
deaths      int64
dtype: object

In [5]:
cases = cases.assign(
    date = pd.to_datetime(cases['date']),
    fips = cases['fips'].astype(pd.Int32Dtype())
)

Cases are cumulative, we want new cases each day to estimate the hospital case load

In [6]:
cases['cases_shifted'] = (
    cases.groupby(['county', 'state'])
    .cases
    .shift(1)
    .fillna(0)
    .astype(int)
)

In [7]:
cases['cases_new'] = cases['cases'] - cases['cases_shifted']

Check logic by plotting King County data.  I've been wanting to see more presentations of new cases per day rather than the cumulative plots that are more commonly displayed to get a better feel for how we are 'flattening the curve'.

In [8]:
king = cases.query('county == "King" & state == "Washington"')

In [9]:
base = alt.Chart(king[['date', 'cases', 'cases_new']]).encode(alt.X('monthdate(date):O', title = 'Date'))
bars = base.mark_bar(color = '#65799b').encode(y = 'cases_new', tooltip = 'cases_new')
bars.properties(title = 'King County WA Epidemiological Curve (new cases per day)')

alt.Chart(...)

In [10]:
line = base.mark_line(color = '#e23e57').encode(y = 'cases', tooltip = 'cases')

(bars + line).properties(title = 'King County Cumulative and Daily Covid-19 Cases')

alt.LayerChart(...)

Passes sanity check.  Interesting that it looks like there are 4 roughly linear regimes on the cumulative curve, just from eyeballing

In [11]:
statewide = (
    cases.groupby(['date', 'state'])
    .aggregate({'cases':'sum'})
    .reset_index()
)

In [12]:
statewide.head()

,date,state,cases
0,2020-01-21,Washington,1
1,2020-01-22,Washington,1
2,2020-01-23,Washington,1
3,2020-01-24,Illinois,1
4,2020-01-24,Washington,1


#### State by State Cases Over Time

In [13]:
(alt.Chart(statewide)
 .mark_rect()
 .encode(x = alt.X('date:O', axis = alt.Axis(labels = False)),
         y = 'state:N',
         color = 'cases:Q')
 .properties(title = 'Cumulative US Covid-19 Cases by Day', width = 600)
)

alt.Chart(...)

In [14]:
(alt.Chart(statewide[statewide['state'] != 'New York'])
 .mark_rect()
 .encode(x = alt.X('date:O', axis = alt.Axis(labels = False)),
         y = 'state:N',
         color = 'cases:Q',
         tooltip = ['date', 'state', 'cases'])
 .properties(title = 'Cumulative US Covid-19 Cases by Day (NY removed)', width = 600)
)

alt.Chart(...)

### Dataset 2 - Population and Demographics by County (U.S. Census Bureau)

In [15]:
pop = pd.read_csv('https://www2.census.gov/programs-surveys/popest/datasets/2010-2017/counties/asrh/cc-est2017-alldata.csv',
                 encoding='ISO-8859-1')

In [16]:
pop.head()

,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,AGEGRP,TOT_POP,TOT_MALE,TOT_FEMALE,...,HWAC_MALE,HWAC_FEMALE,HBAC_MALE,HBAC_FEMALE,HIAC_MALE,HIAC_FEMALE,HAAC_MALE,HAAC_FEMALE,HNAC_MALE,HNAC_FEMALE
0,50,1,1,Alabama,Autauga County,1,0,54571,26569,28002,...,607,538,57,48,26,32,9,11,19,10
1,50,1,1,Alabama,Autauga County,1,1,3579,1866,1713,...,77,56,9,5,4,1,0,0,2,1
2,50,1,1,Alabama,Autauga County,1,2,3991,2001,1990,...,64,66,2,3,2,7,2,3,2,0
3,50,1,1,Alabama,Autauga County,1,3,4290,2171,2119,...,51,57,13,7,5,5,2,1,1,1
4,50,1,1,Alabama,Autauga County,1,4,4290,2213,2077,...,48,44,7,5,0,2,2,1,3,1


#### Join cases and census data to get per-capita infections

In [17]:
# Filter to get most recent census data (2017)
# strip the ' County' from the census data to join to cases
# start with total age

pop17 = (pop
       .query('YEAR == 10 and AGEGRP == 0')
       .assign(county_key = lambda x: x['CTYNAME'].str.extract(r'([A-Za-z\s\.\-]*)(?=\sCounty)'))
       .loc[:,['STNAME', 'CTYNAME', 'county_key', 'TOT_POP']]
      )

In [18]:
# Manually fix a few County/district names so they join to the NYT cases dataset

# All cases for the five boroughs of New York City 
# (New York, Kings, Queens, Bronx and Richmond counties) 
# are assigned to a single area called New York City.
nyc_idx = (pop17.CTYNAME.str.contains(r'New York|Kings|Queens|Bronx|Richmond')) \
    & (pop17.STNAME == 'New York') # and vs & is so confusing to me
    
pop17.loc[nyc_idx, 'county_key'] = 'New York City'

# D.C
pop17.loc[pop17.CTYNAME == 'District of Columbia', 'county_key'] = 'District of Columbia'

In [19]:
pop17.loc[nyc_idx,]

,STNAME,CTYNAME,county_key,TOT_POP
347871,New York,Bronx County,New York City,1471160
351861,New York,Kings County,New York City,2648771
353191,New York,New York County,New York City,1664727
355091,New York,Queens County,New York City,2358582
355471,New York,Richmond County,New York City,479458


In [20]:
pop17_clean = (pop17
               .groupby(['county_key', 'STNAME'])
               .agg({'TOT_POP':'sum'})
               .reset_index()
              )

In [21]:
cases_percap = cases.merge(pop17_clean, how = 'left', left_on = ['county', 'state'], right_on = ['county_key', 'STNAME'])

In [22]:
cases_percap.head()

,date,county,state,fips,cases,deaths,cases_shifted,cases_new,county_key,STNAME,TOT_POP
0,2020-01-21,Snohomish,Washington,53061,1,0,0,1,Snohomish,Washington,801633.0
1,2020-01-22,Snohomish,Washington,53061,1,0,1,0,Snohomish,Washington,801633.0
2,2020-01-23,Snohomish,Washington,53061,1,0,1,0,Snohomish,Washington,801633.0
3,2020-01-24,Cook,Illinois,17031,1,0,0,1,Cook,Illinois,5211263.0
4,2020-01-24,Snohomish,Washington,53061,1,0,1,0,Snohomish,Washington,801633.0


In [23]:
# check mismatches
sum(pd.isnull(cases_percap.TOT_POP))

2267

In [24]:
cases_percap.loc[pd.isnull(cases_percap.TOT_POP), 'county'].value_counts()

Unknown                494
Prince George's         29
Jefferson               29
Virginia Beach city     29
Orleans                 28
                      ... 
Kings                    2
Queens                   2
Bronx                    2
West Carroll             1
New York                 1
Name: county, Length: 120, dtype: int64

Still some mismatches to iron out, but let's go forward with this data for now

In [39]:
cases_percap['cases_per10k'] = cases_percap.cases / cases_percap.TOT_POP * 10000
cases_percap['cases_new_per1k'] = cases_percap.cases_new / cases_percap.TOT_POP * 1000
cases_percap['county_label'] = cases_percap.county + ', ' + cases_percap.STNAME

In [42]:
cases_percap.head()

,date,county,state,fips,cases,deaths,cases_shifted,cases_new,county_key,STNAME,TOT_POP,cases_per10k,cases_new_per1k,county_label
0,2020-01-21,Snohomish,Washington,53061,1,0,0,1,Snohomish,Washington,801633.0,0.012475,0.001247,"Snohomish, Washington"
1,2020-01-22,Snohomish,Washington,53061,1,0,1,0,Snohomish,Washington,801633.0,0.012475,0.000000,"Snohomish, Washington"
2,2020-01-23,Snohomish,Washington,53061,1,0,1,0,Snohomish,Washington,801633.0,0.012475,0.000000,"Snohomish, Washington"
3,2020-01-24,Cook,Illinois,17031,1,0,0,1,Cook,Illinois,5211263.0,0.001919,0.000192,"Cook, Illinois"
4,2020-01-24,Snohomish,Washington,53061,1,0,1,0,Snohomish,Washington,801633.0,0.012475,0.000000,"Snohomish, Washington"


In [53]:
# cumulative cases for counties with > 1 million people
# save data as JSON for better performance with Altair

cases_percap_url = 'cases_percap.json'

cases_percap[cases_percap.TOT_POP > 1000000].to_json(cases_percap_url, orient = 'records')

# can't figure out why the file url isn't working, can read it in just fine with open('cases_percap.json')
# these data are small enough for just saving the dataset with the chart though
cases_percap_big = cases_percap.loc[cases_percap.TOT_POP > 1000000, 
                                    ['date', 'cases_per10k', 'county_label', 'cases_new_per1k', 'STNAME', 'TOT_POP']]

(alt.Chart(cases_percap_big)
    .mark_line(color = 'grey', opacity = 0.6)
    .encode(x = 'date:T',
            y = 'cases_per10k:Q',
            color = 'county_label:N',
            tooltip = 'county_label:N')
    .properties(height = 400, 
                width = 700,
               title = 'Per Capita Covid-19 Cases, Counties > 1M')
    .interactive())

alt.Chart(...)

Getting somewhere, but need a way to visually distinguish. Maybe a panel of plots with cross-filtering?

In [57]:
(alt.Chart(cases_percap_big)
    .mark_point()
    .encode(x = 'TOT_POP:Q',
            y = 'STNAME:N',
            tooltip = 'county_label'))

alt.Chart(...)

In [66]:
# combine the dot plot and the cumulative case plot, with cross-filtering
brush = alt.selection(type = 'interval', resolve = 'global')
base = alt.Chart(cases_percap_big)

pop_points = (base.mark_point()
             .encode(x = alt.X('TOT_POP:Q', axis = alt.Axis(title = 'Population')),
                     y = alt.Y('STNAME:N', axis = alt.Axis(title = 'State')))
             .add_selection(brush)
             .properties(height = 400, 
                         width = 200,
                         title = 'Select County'))

percap_cases_line = (base.mark_line(color = 'grey', opacity = 0.6)
            .encode(x = alt.X('date:T', axis = alt.Axis(title = 'Date')),
                    y = alt.Y('cases_per10k:Q', axis = alt.Axis(title = 'Cases per 10,000 people')),
                    color = 'county_label:N',
                    tooltip = 'county_label:N')
            .properties(height = 400, 
                        width = 500,
                        title = 'Per Capita Covid-19 Cases, Counties > 1M')
            .transform_filter(brush))

pop_points | percap_cases_line

alt.HConcatChart(...)

Wow, Altair is rad!

In [27]:
# Stealing idea that Nate Cohn had for new cases vs prevalence, updating with new data
(alt.Chart(cases_percap.loc[cases_percap.TOT_POP > 2000000, ['cases_per10k', 'cases_new_per1k', 'county']])
    .mark_line()
    .encode(x = 'cases_per10k:Q',
            y = 'cases_new_per1k:Q',
            stroke = 'county:N',
            tooltip = 'county')
    .properties(height = 400, width = 700)
    .interactive())

alt.Chart(...)

Beautiful! Try rolling 3 day avg

### Dataset 3 - Hospitalization Rates by Age Group (CDC)

The CDC published a table of hospitalization, ICU, and death rates by age group, based on ~2500 hospitalizations in the US in Feb/Mar.  This could be used to get a rough estimate of particular types of beds needed, namely pediatric, which is of high interest for us here at Seattle Children's.

In [28]:
# get html from CDC
cdc_html = requests.get('https://www.cdc.gov/mmwr/volumes/69/wr/mm6912e2.htm')
soup = BeautifulSoup(cdc_html.content)

In [29]:
# find table
tables = soup.find_all('table')
tables[0].find_all('tbody')

[<tbody>
 <tr>
 <td align="left" valign="top">0–19 (123)</td>
 <td align="right" valign="bottom">1.6–2.5</td>
 <td align="right" valign="bottom">0</td>
 <td align="right" valign="bottom">0</td>
 </tr>
 <tr>
 <td align="left" valign="top">20–44 (705)</td>
 <td align="right" valign="bottom">14.3–20.8</td>
 <td align="right" valign="bottom">2.0–4.2</td>
 <td align="right" valign="bottom">0.1–0.2</td>
 </tr>
 <tr>
 <td align="left" valign="top">45–54 (429)</td>
 <td align="right" valign="bottom">21.2–28.3</td>
 <td align="right" valign="bottom">5.4–10.4</td>
 <td align="right" valign="bottom">0.5–0.8</td>
 </tr>
 <tr>
 <td align="left" valign="top">55–64 (429)</td>
 <td align="right" valign="bottom">20.5–30.1</td>
 <td align="right" valign="bottom">4.7–11.2</td>
 <td align="right" valign="bottom">1.4–2.6</td>
 </tr>
 <tr>
 <td align="left" valign="top">65–74 (409)</td>
 <td align="right" valign="bottom">28.6–43.5</td>
 <td align="right" valign="bottom">8.1–18.8</td>
 <td align="right" vali

In [30]:
table = tables[0].find('tbody')

rows_dict = dict()

for idx, row in enumerate(table.find_all('tr')):
    temp_list = list()
    for col in row.find_all('td'):
        temp_list.append(col.string)
    rows_dict[idx] = temp_list

In [31]:
cdc_df = pd.DataFrame(rows_dict).T
cdc_df.columns = ['age_range', 'hosp_rate', 'icu_rate', 'death_rate']
cdc_df

,age_range,hosp_rate,icu_rate,death_rate
0,0–19 (123),1.6–2.5,0,0
1,20–44 (705),14.3–20.8,2.0–4.2,0.1–0.2
2,45–54 (429),21.2–28.3,5.4–10.4,0.5–0.8
3,55–64 (429),20.5–30.1,4.7–11.2,1.4–2.6
4,65–74 (409),28.6–43.5,8.1–18.8,2.7–4.9
5,75–84 (210),30.5–58.7,10.5–31.0,4.3–10.5
6,≥85 (144),31.3–70.3,6.3–29.0,10.4–27.3
7,"Total (2,449)",20.7–31.4,4.9–11.5,1.8–3.4


In [32]:
# clean up a bit, extract relevant info into usable columns
# self-challenge by doing this with vectorized pandas functions rather than standard regex

cdc_df = (cdc_df
          .assign(n = lambda x: x['age_range'].str.extract(r'\(([0-9,]{3,})\)'),
                  age_range = lambda x: x['age_range'].str.extract(r'(.*)\s\([0-9,]{3,}\)'),
                  hosp_lo = lambda x: x['hosp_rate'].str.extract(r'([0-9\.]{2,}).[0-9\.]{2,}'),
                  hosp_hi = lambda x: x['hosp_rate'].str.extract(r'[0-9\.]{2,}.([0-9\.]{2,})'))
         )

cdc_df

,age_range,hosp_rate,icu_rate,death_rate,n,hosp_lo,hosp_hi
0,0–19,1.6–2.5,0,0,123,1.6,2.5
1,20–44,14.3–20.8,2.0–4.2,0.1–0.2,705,14.3,20.8
2,45–54,21.2–28.3,5.4–10.4,0.5–0.8,429,21.2,28.3
3,55–64,20.5–30.1,4.7–11.2,1.4–2.6,429,20.5,30.1
4,65–74,28.6–43.5,8.1–18.8,2.7–4.9,409,28.6,43.5
5,75–84,30.5–58.7,10.5–31.0,4.3–10.5,210,30.5,58.7
6,≥85,31.3–70.3,6.3–29.0,10.4–27.3,144,31.3,70.3
7,Total,20.7–31.4,4.9–11.5,1.8–3.4,"2,449",20.7,31.4


In [33]:
# strip off total into separate df
cdc_total = cdc_df[cdc_df.age_range.str.contains('Total')]
cdc_df = cdc_df[~cdc_df.age_range.str.contains('Total')]

### Dataset 4 - Hospital Locations and Capacity (U.S. Homeland Infrastructure Foundation-Level Data)

In [34]:
tables[0].find_all('tr')

[<tr>
 <th align="left" class="row1left" rowspan="2" scope="col" valign="bottom">Age group (yrs) (no. of cases)</th>
 <th align="center" class="row1" colspan="3" scope="col" valign="bottom">%*</th>
 </tr>,
 <tr>
 <th align="center" class="row1" colspan="1" scope="col" valign="bottom">Hospitalization</th>
 <th align="center" class="row1" scope="col" valign="bottom">ICU admission</th>
 <th align="center" class="row1" scope="col" valign="bottom">Case-fatality</th>
 </tr>,
 <tr>
 <td align="left" valign="top">0–19 (123)</td>
 <td align="right" valign="bottom">1.6–2.5</td>
 <td align="right" valign="bottom">0</td>
 <td align="right" valign="bottom">0</td>
 </tr>,
 <tr>
 <td align="left" valign="top">20–44 (705)</td>
 <td align="right" valign="bottom">14.3–20.8</td>
 <td align="right" valign="bottom">2.0–4.2</td>
 <td align="right" valign="bottom">0.1–0.2</td>
 </tr>,
 <tr>
 <td align="left" valign="top">45–54 (429)</td>
 <td align="right" valign="bottom">21.2–28.3</td>
 <td align="right" va